# 🌌 Orbnyt — Autonomous Cognitive Agent System  
### End-to-End Research • Reasoning • Knowledge Graphs • Analytics • Workflows

---

## ⚡ Problem Statement

Modern users expect answers that are **factual**, **multi-step**, and **structured** — not plain LLM text.

Traditional LLMs fail at tasks that require:  
- Reliable web search  
- Multi-source evidence integration  
- Deterministic workflows  
- Knowledge graph construction  
- Dashboard-ready numeric extraction  
- Machine-readable report generation  
- Automated chaining of reasoning steps
- Multi-domain entity extraction (tech products, vehicles, travel, research)
- Qualitative research analysis (benefits/risks/conclusions)
- Safety filtering for harmful content
---

## 🚀 What Orbnyt Does

Orbnyt is a **fully autonomous cognitive agent pipeline** built using the **Google Agent Development Kit (ADK)**.

From a **single natural-language query**, Orbnyt automatically performs:

- 🛡️ **Safety Filtering** — validates queries and blocks unsafe/harmful requests  
- 🔧 **Workflow Self-Correction** — ensures reliable 8-step workflow execution  
- 🌐 **Search** — retrieves authoritative multi-source information  
- 📄 **RAG Synthesis** — compresses and aligns evidence  
- 🔗 **Knowledge Graph Extraction** — generates interpretable triples  
- 🧠 **Multi-Step Workflow Execution** — agents coordinate reasoning with validation  
- 📊 **Analytics + Dashboards** — numeric extraction & visual insights  
- 📝 **Final Report** — clean, human-readable markdown  
- 🧩 **Final JSON Output** — structured, machine-ready data  
- 🎓 **Qualitative Analysis** — extracts benefits, risks, conclusions from research             queries
- 🚗 **Decision Analysis** — cost comparison tables for complex purchase decisions

- 🔄 **Memory Context** — maintains conversation history across 3 previous turns
     This notebook demonstrates Orbnyt as a **complete end-to-end system**, following          enterprise agent design patterns from the Google 5-day Intensive Program.
- 📈 **Auto-Visualization** — generates 5-10 interactive comparison charts automatically
- 📝 **Final Report** — clean, human-readable markdown with structured sections
- 🧩 **Structured Output** — machine-ready data with proper entity tagging

This notebook demonstrates Orbnyt as a **complete end-to-end system**, following enterprise agent design patterns from the Google 5-day Intensive Program.

## 🧠 Orbnyt Architecture Overview

Orbnyt follows a modular **multi-agent cognitive pipeline**, where each stage transforms the user's question into progressively richer structure:

> safety check → workflow planning → self-correction → search → retrieval → compression → knowledge graph → analytics → report

---
```mermaid
flowchart TD

A[User Query] --> SF[🛡️ Safety Filter<br>Query Validation]

SF -->|Safe Query| WP[Workflow Planner<br>8-Step Generation]
WP --> SC_W[Self-Corrector<br>Workflow Validation]
SC_W --> M[Memory Layer<br>Session Context]

M --> B[🌐 Search Agent]
B --> C[📄 RAG Retriever<br>Hybrid Embedding + TF-IDF]

C --> S[Summarizer<br>Evidence Compression]
S --> K[🔗 Knowledge Graph Extractor<br>JSON Triples]

K --> AN[Analyzer<br>Structured Insights]

AN --> SC_A{🔧 Self-Correction Check<br>Data Quality Validation}

SC_A -->|Data Valid| A1[📊 Analytics Engine<br>Pandas + Plotly]
SC_A -->|Data Invalid| K

A1 --> R[📝 Final Report Generator<br>Clean Markdown]
R --> J[🧩 Final JSON Builder<br>Structured Output]

J --> M

SF -->|Unsafe Query| BLOCK[❌ Blocked<br>Safety Response]

In [ ]:
# ============================================================
# Orbnyt — Core Environment Setup (Clean)
# ============================================================

import os
from kaggle_secrets import UserSecretsClient

# -------------------------------
# 1. Load Google API Key
# -------------------------------
GOOGLE_API_KEY = None
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except KeyError as e:
    raise ValueError("GOOGLE_API_KEY is missing in Kaggle Secrets.") from e

# -------------------------------
# 2. ADK Core Imports
# -------------------------------
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LlmAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, FunctionTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

# -------------------------------
# 3. Retry Configuration
# -------------------------------
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# -------------------------------
# 4. Session Memory
# -------------------------------
session_service = InMemorySessionService()
SESSION_ID = "orbnyt_global_session"

# -------------------------------
# 5. Plotly for Dashboards
# -------------------------------
import plotly.graph_objects as go
import plotly.express as px

# -------------------------------
# 6. HTML Utilities
# -------------------------------
from IPython.display import HTML, display

# -------------------------------
# 7. Final Silent Ready Notice
# -------------------------------
print("🌌 Orbnyt environment initialized.")

In [ ]:
# ============================================================
# CELL 4 — Search Agent
# ============================================================

search_agent = Agent(
    name="SearchAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    tools=[google_search],
    instruction="""
Use ONLY the google_search tool.
Return a concise, factual summary based strictly on retrieved snippets.

IMPORTANT:
Always include ALL numeric specifications available:
- Battery (mAh)
- Charging speed (W)
- Camera (MP)
- Display size (inches)
- CPU clock (GHz)
- RAM/Storage (GB/TB)
- Refresh rate (Hz)
- Price ($, €, ₹)

Never omit numeric specs.


Hard constraints:
- No speculation or assumptions.
- No model opinions or guesses.
- No filler text.
- Do NOT mention sources, URLs, or tool usage.
- Output clean factual text only.
""",
    output_key="search_output", 
)

search_runner = InMemoryRunner(agent=search_agent)

In [ ]:
# ============================================================
# CELL 5 — StrongRAG (Hybrid: Embeddings + TF-IDF)
# ============================================================

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from google.genai import Client
import os

client = Client(api_key=os.environ["GOOGLE_API_KEY"])

# -------------------------------------------------------------
# 1) Embedding Function 
# -------------------------------------------------------------
def embed_texts(texts):
    """
    texts: list[str]
    returns: np.array of vectors
    """
    payload = [{"parts": [{"text": t}]} for t in texts]
    
    resp = client.models.embed_content(
        model="models/text-embedding-004",
        contents=payload
    )
    
    vectors = [e.values for e in resp.embeddings]
    return np.array(vectors, dtype=float)


# -------------------------------------------------------------
# 2) Chunk Text 
# -------------------------------------------------------------
def chunk_text(text, chunk_size=350):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size]).strip()
        if chunk:
            chunks.append({"id": len(chunks), "text": chunk})
    return chunks


# -------------------------------------------------------------
# 3) TF-IDF Retriever 
# -------------------------------------------------------------
class TfidfRetriever:
    def __init__(self, chunks):
        texts = [c["text"] for c in chunks]
        self.chunks = chunks
        self.vectorizer = TfidfVectorizer(stop_words="english")
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)

    def query(self, question):
        q_vec = self.vectorizer.transform([question])
        scores = (q_vec @ self.tfidf_matrix.T).toarray()[0]
        
        ranked = [
            {"id": c["id"], "text": c["text"], "tfidf_score": float(scores[i])}
            for i, c in enumerate(self.chunks)
        ]
        ranked.sort(key=lambda x: x["tfidf_score"], reverse=True)
        return ranked


# -------------------------------------------------------------
# 4) StrongRAG
# -------------------------------------------------------------
class StrongRAG:
    def __init__(self):
        self.chunks = []
        self.embeddings = None
        self.tfidf = None

    def index(self, text, memory_text=None):
        # merge memory (optional)
        if memory_text:
            text = f"{memory_text}\n\n{text}"
        
        # chunk, embed, tfidf
        self.chunks = chunk_text(text)
        self.tfidf = TfidfRetriever(self.chunks)
        
        chunk_texts = [c["text"] for c in self.chunks]
        self.embeddings = embed_texts(chunk_texts)

    def retrieve(self, question, top_k=5):
        top_k = max(1, min(top_k, len(self.chunks)))
        
        # embedding score
        q_vec = embed_texts([question])[0]
        
        def cos(a, b):
            denom = np.linalg.norm(a) * np.linalg.norm(b)
            return float(np.dot(a, b) / denom) if denom > 0 else 0.0
        
        embed_scores = np.array([cos(q_vec, e) for e in self.embeddings], dtype=float)
        
        # tf-idf score
        tfidf_rank = self.tfidf.query(question)
        tfidf_scores = np.array([r["tfidf_score"] for r in tfidf_rank], dtype=float)
        
        # normalize silently
        if embed_scores.max() > 0:
            embed_scores /= embed_scores.max()
        if tfidf_scores.max() > 0:
            tfidf_scores /= tfidf_scores.max()
        
        # combined
        combined = 0.65 * embed_scores + 0.35 * tfidf_scores
        
        # package results
        results = []
        for i, c in enumerate(self.chunks):
            results.append({
                "id": c["id"],
                "text": c["text"],
                "embed_score": float(embed_scores[i]),
                "tfidf_score": float(tfidf_scores[i]),
                "combined_score": float(combined[i])
            })
        
        results.sort(key=lambda x: x["combined_score"], reverse=True)
        return results[:top_k]


# -------------------------------------------------------------
# 5) Instantiate RAG 
# -------------------------------------------------------------
rag = StrongRAG()

In [ ]:
# ============================================================
# CELL 6 — Knowledge Graph Extractor
# ============================================================

import json
import re

# ------------------------------------------------------------
# 1. KG Agent
# ------------------------------------------------------------
kg_agent = Agent(
    name="KnowledgeGraphExtractor",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
Extract ALL factual relationships as JSON triples.

Output format (STRICT):
[
  {"subject": "iPhone 16", "predicate": "has_battery", "object": "3561 mAh"},
  {"subject": "iPhone 16", "predicate": "has_display", "object": "6.1 inch"},
  {"subject": "iPhone 16", "predicate": "powered_by", "object": "A18 chip"},
  {"subject": "Pixel 9", "predicate": "has_battery", "object": "4700 mAh"},
  {"subject": "Pixel 9", "predicate": "has_camera", "object": "50MP"}
]

Rules:
- Extract at LEAST 20 triples if text is long
- Focus on: specifications, features, comparisons, relationships
- Use simple predicates: "has_", "is_", "supports_", "features_"
- NO markdown, NO code fences, NO commentary
- ONLY the JSON array

If comparing two entities, extract triples for BOTH.
""",
    output_key="triples"
)

kg_runner = InMemoryRunner(agent=kg_agent)

# ------------------------------------------------------------
# 2. Triple Cleaner
# ------------------------------------------------------------
def _clean_triple(t):
    return {
        "subject": str(t.get("subject", "")).strip(),
        "predicate": str(t.get("predicate", "")).strip(),
        "object": str(t.get("object", "")).strip(),
    }

# ------------------------------------------------------------
# 3. JSON Array Extractor
# ------------------------------------------------------------
def _extract_json_array(text):
    if not text or not isinstance(text, str):
        return None
    
    cleaned = text.replace("``````", "").strip()
    
    try:
        data = json.loads(cleaned)
        if isinstance(data, list):
            return data
        if isinstance(data, dict) and "triples" in data:
            return data["triples"]
    except:
        pass
    
    # Regex fallback
    try:
        match = re.search(r'.∗?.*?', cleaned, re.DOTALL)
        if match:
            return json.loads(match.group(0))
    except:
        pass

    return None

# ------------------------------------------------------------
# 4. Enhanced KG Extract Function 
# ------------------------------------------------------------
async def extract_kg_triples(text):
    """
    Extract KG triples with enhanced prompting and error handling.
    Always return a list (possibly empty).
    """
    if not text:
        return []

    # fix for long text
    if len(text) > 8000:
        text = text[:8000] + " ..."

    prompt = (
        "Extract factual triples from this text. "
        "Return ONLY a JSON array of objects with keys: subject, predicate, object.\n\n"
        f"Text:\n{text}"
    )

    print(f"DEBUG extract_kg_triples: sending {len(prompt)} chars to KG agent")

    try:
        with SilentOutput():
            resp = await kgrunner.run_debug(prompt)
    except Exception:
        return []

    raw = extract_llm_text(resp)
    print(f"DEBUG KG response: got {len(raw)} chars back")

    arr = extract_json_array(raw)
    if not arr:
        print("WARNING: Could not parse JSON from KG agent response!")
        return []

    print(f"Parsed {len(arr)} triples from JSON")
    return [clean_triplet(t) for t in arr]
print ("🤖knowledge graph executor added succesfully")

In [ ]:
# ============================================================
# CELL 7 — Workflow Composer 
# ============================================================

import json
import re
from google.adk.models.google_llm import Gemini
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner

# ------------------------------------------------------------
# 0. Safety Filter (Blocks Unsafe Queries)
# ------------------------------------------------------------
UNSAFE_KEYWORDS = [
    "suicide", "kill myself", "how to die", "self harm",
    "bomb", "make a bomb", "terrorist", "harm someone",
    "hack", "bypass", "illegal", "crime", "drugs",
    "weapon", "shoot", "murder", "rape", "abuse"
]

def is_unsafe_query(text: str) -> bool:
    """Check if query contains unsafe keywords."""
    t = text.lower()
    return any(k in t for k in UNSAFE_KEYWORDS)

def safety_response():
    """Return safety block response."""
    return {
        "safe": False,
        "message": "I cannot help with this request. Let me know if you want anything else."
    }

def strip_fences(text: str):
    """Remove markdown code fences."""
    if not text:
        return ""
    return text.replace("`````````", "").strip()


# ------------------------------------------------------------
# Allowed actions
# ------------------------------------------------------------
ALLOWED_ACTIONS = [
    "search",
    "rag_retrieve",
    "summarize",
    "extract_kg",
    "analyze_text",
    "generate_dashboard",
    "final_report"
]

# ------------------------------------------------------------
# Workflow Planner Agent
# ------------------------------------------------------------
workflow_planner_agent = Agent(
    name="workflow_planner",
    model=Gemini(model="gemini-2.5-flash"),
    instruction=(
        "You output ONLY valid JSON (no markdown, no prose).\n"
        "Your job: Convert the user question into a SAFE and CORRECT workflow.\n\n"
        "Input will be a JSON object of the form:\n"
        "{\"current_question\": \"...\", \"previous_context\": \"...\"}\n\n"
        "Use previous_context only to maintain continuity across turns, "
        "but ALWAYS answer the current_question explicitly.\n\n"
        "ALLOWED_ACTIONS = " + json.dumps(ALLOWED_ACTIONS, indent=2) + "\n\n"
        "Output a JSON LIST where each element is {\"action\": ..., \"input\": ...}.\n"
        "Use '$prev' to pass output from the previous step.\n"
        "Do not include unsafe tools.\n"
        "Remember: JSON only."
    ),
    output_key="steps",
)

workflow_planner_runner = InMemoryRunner(agent=workflow_planner_agent)

# ------------------------------------------------------------
# Self-correction agent
# ------------------------------------------------------------
self_correct_agent = Agent(
    name="WorkflowCorrector",
    model=Gemini(model="gemini-2.5-flash"),
    instruction="""
You receive: {"original_query": "...", "steps": [...]}

Return a SAFE, CLEAN workflow as a JSON LIST of steps.
Each step must be:
{"action": "<one of: search, summarize, extract_kg, generate_dashboard, final_report>",
 "input": "<string or $prev>"}

FOR THIS DEMO, YOU MUST RETURN EXACTLY THESE 5 STEPS IN ORDER:

1. {"action": "search", "input": "<original_query>"}
2. {"action": "summarize", "input": "$prev"}
3. {"action": "extract_kg", "input": "$prev"}
4. {"action": "generate_dashboard", "input": "$prev"}
5. {"action": "final_report", "input": "$prev"}

Output JSON only. No commentary.
""",
    output_key="fixed"
)

self_correct_runner = InMemoryRunner(agent=self_correct_agent)

# ------------------------------------------------------------
# Workflow planning
# ------------------------------------------------------------
async def plan_workflow(question: str, previous_context: str = ""):
    if is_unsafe_query(question):
        return safety_response()
    
    # Prepare structured input
    planner_input = {
        "current_question": question,
        "previous_context": previous_context or ""
    }
    
    # RUN PLANNER
    resp = await workflow_planner_runner.run_debug(json.dumps(planner_input))
    
    raw_json = None
    
    # Direct output attributes
    if hasattr(resp, "output") and resp.output:
        try:
            cleaned = strip_fences(str(resp.output))
            raw_json = json.loads(cleaned)
        except:
            pass
    
    if raw_json is None and hasattr(resp, "output_text") and resp.output_text:
        try:
            cleaned = strip_fences(str(resp.output_text))
            raw_json = json.loads(cleaned)
        except:
            pass
    
    # Extract from string representation
    if raw_json is None:
        raw_str = str(resp)
        match = re.search(r'\[\s*\{.*?\}\s*\]', raw_str, re.DOTALL)
        if match:
            try:
                raw_json = json.loads(match.group(0))
            except:
                pass
    
    # basic 5-step workflow
    if raw_json is None:
        raw_json = [
            {"action": "search", "input": question},
            {"action": "summarize", "input": "$prev"},
            {"action": "extract_kg", "input": "$prev"},
            {"action": "generate_dashboard", "input": "$prev"},
            {"action": "final_report", "input": "$prev"}
        ]
    
    # --------------------------------------------------------
    # Self-correct the workflow into EXACT 5-step demo pipeline
    # --------------------------------------------------------
    payload = {"original_query": question, "steps": raw_json}
    fix_resp = await self_correct_runner.run_debug(json.dumps(payload))
    
    fixed = None
    
    if hasattr(fix_resp, "output") and fix_resp.output:
        try:
            fixed = json.loads(strip_fences(str(fix_resp.output)))
        except:
            pass
    
    if fixed is None and hasattr(fix_resp, "output_text") and fix_resp.output_text:
        try:
            fixed = json.loads(strip_fences(str(fix_resp.output_text)))
        except:
            pass
    
    if fixed is None:
        raw = str(fix_resp)
        match = re.search(r'\s∗{.∗?}\s∗\s*\{.*?\}\s*', raw, re.DOTALL)
        if match:
            try:
                fixed = json.loads(match.group(0))
            except:
                pass
    
    # Final fallback
    if fixed is None:
        fixed = [
            {"action": "search", "input": question},
            {"action": "summarize", "input": "$prev"},
            {"action": "extract_kg", "input": "$prev"},
            {"action": "generate_dashboard", "input": "$prev"},
            {"action": "final_report", "input": "$prev"}
        ]
    
    return fixed

# ------------------------------------------------------------
# Sanitization & compilation
# ------------------------------------------------------------
def sanitize_workflow(steps):
    clean = []
    for s in steps:
        if "action" in s and s["action"] in ALLOWED_ACTIONS:
            if "input" not in s or not str(s["input"]).strip():
                s["input"] = "$prev"
            clean.append(s)
    return clean


def compile_workflow(steps):
    return [{"action": s["action"], "input": s["input"]} for s in steps]


print("🚀 Workflow Composer loaded")

In [ ]:
# ============================================================
# Analyzer Agent + Final Report Agent + Summarizer
# ============================================================

from google.adk.models.google_llm import Gemini
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

# ---------------------------------------------------------
# Retry Configuration 
# ---------------------------------------------------------
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# ---------------------------------------------------------
# Summarizer
# ---------------------------------------------------------
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
Summarize the provided text FACTUALLY while PRESERVING ALL NUMERIC DETAILS.

Rules:
- Do NOT remove any numbers, units, or measurements (mAh, MP, Hz, W, $, €, ₹, inches).
- Preserve factual attributes exactly as stated.
- No markdown.
- No bullet points.
- No opinions.
- No added knowledge.
- No hallucinations.
- don't remove any necessary parts in search output
- summarize very minimal
- Only compress and reorganize what is given.
- Maintain all comparisons and device-specific specifications.
""",
    output_key="summary_output"
)

summarizer_runner = InMemoryRunner(agent=summarizer_agent)

# ---------------------------------------------------------
# Analyzer
# ---------------------------------------------------------
analyzer_agent = Agent(
    name="AnalyzerAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
Analyze ONLY the facts in the input.

Rules:
- No hallucination
- Use ONLY input text
- Identify entities, attributes, and numeric values
- If comparing two entities, include comparison sections
- Markdown allowed

Output pattern:
If two entities (A vs B):
  ## A vs B — Factual Analysis
  ### A — Key Findings
  - ...
  ### B — Key Findings
  - ...
  ### Comparison
  - Similarities
  - Differences
  ### Numeric Table
  | Metric | A | B |

If one entity:
  ## Entity Overview
  ### Key Attributes
  - ...
  ### Numeric Facts
  - ...
""",
    output_key="analysis_output"
)

analyzer_runner = InMemoryRunner(agent=analyzer_agent)

# ---------------------------------------------------------
# Final Report Agent
# ---------------------------------------------------------
final_report_agent = Agent(
    name="FinalReportAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
Write a structured enterprise report using ONLY the provided text.

Sections:
# Final Report

## 1. Executive Summary
## 2. Entity Overviews
## 3. Comparison Table (if applicable)
## 4. Knowledge Graph Insights
## 5. Numeric Insights
## 6. Analytical Interpretation
## 7. Conclusion

Rules:
- Markdown only
- No hallucination
- No new facts
- If missing data, write: "No information provided."
""",
    output_key="report_output"
)

final_report_runner = InMemoryRunner(agent=final_report_agent)

In [ ]:
# ============================================================
# Workflow Executor 
# ============================================================

import json
import re
import pandas as pd
import plotly.express as px
import asyncio
import sys
import io

# ---------------------------------------------
# Silent output helper
# ---------------------------------------------
class SilentOutput:
    """Suppresses all print output inside a with-block."""
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = io.StringIO()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout


# ---------------------------------------------
# Utility: normalize previous output
# ---------------------------------------------
def normalize_prev(p):
    if p is None:
        return ""
    if isinstance(p, pd.DataFrame):
        return p.to_csv(index=False)
    if isinstance(p, (dict, list)):
        try:
            return json.dumps(p)
        except Exception:
            return str(p)
    return re.sub(r"\s+", " ", str(p).strip())


# ---------------------------------------------
# Clean LLM text
# ---------------------------------------------

def extract_llm_text(resp):
    """Extract clean text from Gemini response, remove metadata - AGGRESSIVE."""
    if hasattr(resp, "output_text") and resp.output_text:
        text = resp.output_text
    elif hasattr(resp, "output") and resp.output:
        text = resp.output
    elif hasattr(resp, "text"):
        text = resp.text
    else:
        text = str(resp)
    
    # Try to extract from Event list format
    if isinstance(text, str) and text.startswith('['):
        # Look for text='...' pattern
        match = re.search(r"text='(.*?)'(?:\s*,|\s*\))", text, re.DOTALL)
        if match:
            content = match.group(1)
            if len(content) > 50:
                return content
    
    # Remove Event(...) wrappers
    text = re.sub(r"Event\(.*?content=Content\(.*?text='(.*?)'.*?\)", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"Event\(.*?\)", "", text, flags=re.DOTALL)
    
    # Remove grounding / usage / action metadata
    text = re.sub(r"grounding_metadata.*?(?=,\s*\w+[A-Z]|\)|$)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"usage_metadata.*?(?=,\s*\w+[A-Z]|\)|$)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"actions=.*?(?=,\s*\w+[A-Z]|\)|$)", "", text, flags=re.DOTALL)
    
    # Remove URLs
    text = re.sub(r"https?://\S+", "", text)
    
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text.strip())
    return text
# ---------------------------------------------
# MULTI-DOMAIN NUMBER EXTRACTOR
# ---------------------------------------------
def extract_numbers_from_text(t, question=""):
    """
    Dynamic entity extractor based on question context.
    Strategy:
      1. Detect entities from question keywords.
      2. Segment text into blocks per entity based on name occurrences.
      3. Extract numbers inside each block and tag them with that entity.
      4. Fallback: line‑by‑line with last‑seen entity context.
    """
    nums = []
    text = t if isinstance(t, str) else str(t)
    
    # ---------------------------------------------
    # 1. Detect entities from question
    # ---------------------------------------------
    q_lower = question.lower()
    
    # Tech comparison keywords
    if any(x in q_lower for x in ["iphone", "pixel", "phone", "smartphone"]):
        entities = ["Iphone 16", "Pixel 9"]
        patterns = {
            "Iphone 16": re.compile(r"\biphone\s*16\b", re.I),
            "Pixel 9": re.compile(r"\b(?:google\s+)?pixel\s*9\b", re.I),
        }
    # Vehicle comparison keywords
    elif any(x in q_lower for x in ["scooter", "car", "vehicle", "commute", "electric", "fuel"]):
        entities = ["Electric Scooter", "Used Car"]
        patterns = {
            "Electric Scooter": re.compile(r"\b(?:electric\s+)?(?:e-)?scooter|ev\s+scooter|two-?wheeler|ather|ola\b", re.I),
            "Used Car": re.compile(r"\b(?:used\s+)?(?:small\s+)?car|vehicle|maruti|hyundai|alto|swift|wagon\s*r|petrol|cng\b", re.I),
        }
    # Travel / trip planning keywords
    elif any(x in q_lower for x in ["tokyo", "paris", "trip", "travel", "vacation", "plan", "flight", "hotel"]):
        entities = ["Flights", "Accommodation", "Food", "Transport", "Activities"]
        patterns = {
            "Flights": re.compile(r"\b(?:flight|airline|airfare|air\s+ticket)\b", re.I),
            "Accommodation": re.compile(r"\b(?:hotel|hostel|accommodation|lodging|stay|capsule|sharehouse)\b", re.I),
            "Food": re.compile(r"\b(?:food|meal|restaurant|vegan|dining|eating)\b", re.I),
            "Transport": re.compile(r"\b(?:transport|metro|train|taxi|bus|travel\s+pass|jr\s+pass)\b", re.I),
            "Activities": re.compile(r"\b(?:temple|museum|attraction|ticket|entry|visit)\b", re.I),
        }
    # Research/qualitative topics (AI in education, etc.) — skip numeric extraction
    elif any(x in q_lower for x in ["generative ai", "education", "benefits", "risks", "accuracy", "cheating", "accessibility"]):
        return []  # No numeric extraction for qualitative research queries
    # Generic fallback
    else:
        entities = ["Item"]
        patterns = {"Item": re.compile(r".+", re.I)}
    
    # ---------------------------------------------
    # 2. Find anchor positions for each entity
    # ---------------------------------------------
    anchors = []
    for ent, pat in patterns.items():
        for m in pat.finditer(text):
            anchors.append({"entity": ent, "start": m.start()})
    anchors.sort(key=lambda x: x["start"])
    
    # If no anchors found, fall back to line‑based context
    if not anchors:
        return _extract_with_line_context(text, entities)
    
    # ---------------------------------------------
    # 3. Build blocks:per anchor
    # ---------------------------------------------
    blocks = []
    for i, a in enumerate(anchors):
        start = a["start"]
        end = anchors[i + 1]["start"] if i + 1 < len(anchors) else len(text)
        blocks.append({"entity": a["entity"], "text": text[start:end]})
    
    # ---------------------------------------------
    # 4. Extract numbers per block
    # ---------------------------------------------
    for block in blocks:
        ent = block["entity"]
        btext = block["text"]
        lines = btext.split("\n")
        
        for line in lines:
            # -------- BATTERY (mAh) --------
            for m in re.findall(r"(\d{3,5})\s?mAh", line, re.I):
                nums.append({"entity": ent, "metric": "battery_mAh", "value": float(m), "unit": "mAh"})
            
            # -------- PRICE / COST --------
            for m in re.findall(r"[$€£¥₹]\s?[\d,]+", line):
                clean = float(m[1:].replace(",", "").replace(" ", ""))
                if 10 < clean < 200000:  # broader range for travel/vehicle costs
                    nums.append({"entity": ent, "metric": "price", "value": clean, "unit": "currency"})
            
            # -------- CAMERA (MP) --------
            for m in re.findall(r"(\d{1,3})\s?MP", line, re.I):
                nums.append({"entity": ent, "metric": "camera_MP", "value": float(m), "unit": "MP"})
            
            # -------- CHARGING (W) --------
            for m in re.findall(r"(\d{1,3})\s?W\b", line, re.I):
                nums.append({"entity": ent, "metric": "charging_W", "value": float(m), "unit": "W"})
            
            # -------- REFRESH RATE (Hz) --------
            for m in re.findall(r"(\d{2,3})\s?Hz", line, re.I):
                nums.append({"entity": ent, "metric": "refresh_Hz", "value": float(m), "unit": "Hz"})
            
            # -------- CPU (GHz) --------
            for m in re.findall(r"(\d\.\d{1,2})\s?GHz", line, re.I):
                nums.append({"entity": ent, "metric": "cpu_GHz", "value": float(m), "unit": "GHz"})
            
            # -------- STORAGE (GB) --------
            for m in re.findall(r"(\d{1,3})\s?GB", line, re.I):
                val = float(m)
                if 1 <= val <= 2048:
                    nums.append({"entity": ent, "metric": "capacity_GB", "value": val, "unit": "GB"})
            
            # -------- DISPLAY SIZE (inch) --------
            for m in re.findall(r"(\d\.\d{1,2})\s?inch", line, re.I):
                nums.append({"entity": ent, "metric": "display_inches", "value": float(m), "unit": "inch"})
            
            # -------- DAYS (for trip duration) --------
            for m in re.findall(r"(\d{1,2})\s?(?:day|night)s?", line, re.I):
                val = float(m)
                if 1 <= val <= 90:
                    nums.append({"entity": ent, "metric": "duration_days", "value": val, "unit": "days"})
            
            # -------- KM/MILEAGE --------
            for m in re.findall(r"(\d{1,3})\s?km(?:/l)?", line, re.I):
                val = float(m)
                if 1 <= val <= 500:
                    nums.append({"entity": ent, "metric": "mileage_km", "value": val, "unit": "km"})
    
    # If somehow only one entity got values in multi-entity mode, fall back to line‑context mode
    if len(entities) > 1 and len({n["entity"] for n in nums}) < 2:
        return _extract_with_line_context(text, entities)
    
    return nums


def _extract_with_line_context(text, entities):
    """
    Fallback: line‑by‑line extraction with last‑seen entity context 
    using explicit name mentions in each line.
    """
    nums = []
    lines = text.split("\n")
    current_entity = entities[0] if entities else "Item"
    
    # Build dynamic patterns for all entities
    entity_patterns = {}
    for ent in entities:
        if ent == "Iphone 16":
            entity_patterns[ent] = re.compile(r"\biphone\s*16\b", re.I)
        elif ent == "Pixel 9":
            entity_patterns[ent] = re.compile(r"\b(?:google\s+)?pixel\s*9\b", re.I)
        elif ent == "Electric Scooter":
            entity_patterns[ent] = re.compile(r"\b(?:electric\s+)?scooter|e-scooter|ev|two-?wheeler|ather|ola\b", re.I)
        elif ent == "Used Car":
            entity_patterns[ent] = re.compile(r"\b(?:used\s+)?(?:small\s+)?car|vehicle|maruti|hyundai|alto|swift|petrol|cng\b", re.I)
        elif ent == "Flights":
            entity_patterns[ent] = re.compile(r"\b(?:flight|airline|airfare)\b", re.I)
        elif ent == "Accommodation":
            entity_patterns[ent] = re.compile(r"\b(?:hotel|hostel|accommodation|stay)\b", re.I)
        elif ent == "Food":
            entity_patterns[ent] = re.compile(r"\b(?:food|meal|restaurant|vegan|dining)\b", re.I)
        elif ent == "Transport":
            entity_patterns[ent] = re.compile(r"\b(?:transport|metro|train|taxi|bus)\b", re.I)
        elif ent == "Activities":
            entity_patterns[ent] = re.compile(r"\b(?:temple|museum|attraction|visit)\b", re.I)
        else:
            entity_patterns[ent] = re.compile(r".+", re.I)
    
    for line in lines:
        lower = line.lower()
        
        # Update current entity based on line content
        for ent, pat in entity_patterns.items():
            if pat.search(line):
                current_entity = ent
                break
        
        # -------- PRICE --------
        for m in re.findall(r"[$€£¥₹]\s?[\d,]+", line):
            clean = float(m[1:].replace(",", "").replace(" ", ""))
            if 10 < clean < 200000:
                nums.append({"entity": current_entity, "metric": "price", "value": clean, "unit": "currency"})
        
        # -------- BATTERY (mAh) --------
        for m in re.findall(r"(\d{3,5})\s?mAh", line, re.I):
            nums.append({"entity": current_entity, "metric": "battery_mAh", "value": float(m), "unit": "mAh"})
        
        # -------- CAMERA (MP) --------
        for m in re.findall(r"(\d{1,3})\s?MP", line, re.I):
            nums.append({"entity": current_entity, "metric": "camera_MP", "value": float(m), "unit": "MP"})
        
        # -------- CHARGING (W) --------
        for m in re.findall(r"(\d{1,3})\s?W\b", line, re.I):
            nums.append({"entity": current_entity, "metric": "charging_W", "value": float(m), "unit": "W"})
        
        # -------- REFRESH RATE (Hz) --------
        for m in re.findall(r"(\d{2,3})\s?Hz", line, re.I):
            nums.append({"entity": current_entity, "metric": "refresh_Hz", "value": float(m), "unit": "Hz"})
        
        # -------- CPU (GHz) --------
        for m in re.findall(r"(\d\.\d{1,2})\s?GHz", line, re.I):
            nums.append({"entity": current_entity, "metric": "cpu_GHz", "value": float(m), "unit": "GHz"})
        
        # -------- STORAGE (GB) --------
        for m in re.findall(r"(\d{1,3})\s?GB", line, re.I):
            val = float(m)
            if 1 <= val <= 2048:
                nums.append({"entity": current_entity, "metric": "capacity_GB", "value": val, "unit": "GB"})
        
        # -------- DISPLAY SIZE (inch) --------
        for m in re.findall(r"(\d\.\d{1,2})\s?inch", line, re.I):
            nums.append({"entity": current_entity, "metric": "display_inches", "value": float(m), "unit": "inch"})
        
        # -------- DAYS --------
        for m in re.findall(r"(\d{1,2})\s?(?:day|night)s?", line, re.I):
            val = float(m)
            if 1 <= val <= 90:
                nums.append({"entity": current_entity, "metric": "duration_days", "value": val, "unit": "days"})
    
    return nums


# ---------------------------------------------
# Dashboard wrapper
# ---------------------------------------------
def generate_dashboard(metrics):
    return {"status": "ready_for_plotting", "count": len(metrics), "sample": metrics[:5]}


# ---------------------------------------------
# run_step 
# ---------------------------------------------
async def run_step(action, inp, prev, memory_blob, search_outputs, question=""):
    # Throttle to reduce per-minute limit
    await asyncio.sleep(1.2)
    p = normalize_prev(prev)
    
    try:
        if action == "search":
            query = inp.replace("$prev", p)
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    with SilentOutput():
                        resp = await search_runner.run_debug(query)
                    clean = extract_llm_text(resp)
                    if clean and len(clean) > 50:
                        search_outputs.append(clean)
                        return clean
                    if attempt < max_retries - 1:
                        await asyncio.sleep(1.5)
                except Exception:
                    if attempt < max_retries - 1:
                        await asyncio.sleep(1.5)
            return ""
        
        if action == "summarize":
            with SilentOutput():
                resp = await summarizer_runner.run_debug(p)
            return extract_llm_text(resp)
        
        if action == "rag_retrieve":
            if search_outputs:
                rag.index(" ".join(search_outputs), memory_blob)
            q = p if p.strip() else "comparison analysis"
            r = rag.retrieve(q, top_k=5)
            return " ".join([x["text"] for x in r]) if r else p
        
        if action == "extract_kg":
            full_text = " ".join(search_outputs) if search_outputs else p
            with SilentOutput():
                triples = await extract_kg_triples(full_text)
            return triples
        
        if action == "analyze_text":
            with SilentOutput():
                resp = await analyzer_runner.run_debug(p)
            return extract_llm_text(resp)
        
        if action == "generate_dashboard":
            base = " ".join(search_outputs) if search_outputs else p
            metrics = extract_numbers_from_text(base, question=question)
            return {"metrics": metrics, "dashboard": generate_dashboard(metrics)}
        
        if action == "final_report":
            full = json.dumps(prev, indent=2)
            with SilentOutput():
                resp = await final_report_runner.run_debug(full)
            return extract_llm_text(resp)
    
    except Exception as e:
        return f"Error in {action}: {e}"


# ---------------------------------------------
# execute_workflow
# ---------------------------------------------
async def execute_workflow(steps, memory_blob="", question=""):
    prev = ""
    search_outputs = []
    out = {
        "search": "",
        "rag": "",
        "summarize": "",
        "extract_kg": [],
        "analyze_text": "",
        "generate_dashboard": {},
        "final_report": "",
        "question": question
    }
    
    for step in steps:
        action = step["action"]
        inp = step["input"]
        prev = await run_step(action, inp, prev, memory_blob, search_outputs, question=question)
        out[action] = prev
    
    gd = out.get("generate_dashboard")
    if isinstance(gd, dict) and "metrics" in gd:
        try:
            out["raw_table"] = pd.DataFrame(gd["metrics"])
        except Exception:
            out["raw_table"] = None
    
    return out


print("✅ Workflow Executor Loaded")

In [ ]:
# ============================================================
# Research Report Engine
# ============================================================

import json
import networkx as nx
import plotly.graph_objects as go

# 1. Knowledge Graph Visualizer
def visualize_kg_graph(triples):
    """Build a graph from KG triples and return a Plotly Figure."""
    G = nx.DiGraph()
    
    for t in triples:
        try:
            s = t.get("subject")
            p = t.get("predicate")
            o = t.get("object")
            if s and o:
                G.add_node(s)
                G.add_node(o)
                G.add_edge(s, o, label=p)
        except Exception:
            continue
    
    if G.number_of_nodes() == 0:
        return go.Figure()
    
    pos = nx.spring_layout(G, seed=42, k=0.55)
    x_nodes = [pos[n][0] for n in G.nodes]
    y_nodes = [pos[n][1] for n in G.nodes]
    node_labels = list(G.nodes)
    
    x_edges = []
    y_edges = []
    for s, o in G.edges:
        x_edges += [pos[s][0], pos[o][0], None]
        y_edges += [pos[s][1], pos[o][1], None]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x_edges, y=y_edges,
        mode="lines",
        line=dict(width=1, color="gray"),
        hoverinfo="none"
    ))
    fig.add_trace(go.Scatter(
        x=x_nodes, y=y_nodes,
        mode="markers+text",
        text=node_labels,
        textposition="bottom center",
        marker=dict(size=16, color="lightblue", line=dict(width=2, color="darkblue")),
        hoverinfo="text"
    ))
    fig.update_layout(
        title="Knowledge Graph",
        showlegend=False,
        margin=dict(l=10, r=10, t=40, b=10),
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        width=800,
        height=600
    )
    return fig


# 2. Memory
conversation_memory = []


# 3. Build KG directly from dashboard_table
def build_kg_from_table(raw_table):
    """
    Build simple KG triples directly from dashboard_table.
    subject   = entity name (e.g., 'Iphone 16')
    predicate = 'has_<metric>'
    object    = '<value> <unit>'
    """
    import pandas as pd  

    if raw_table is None or isinstance(raw_table, str):
        return []
    if not isinstance(raw_table, pd.DataFrame) or raw_table.empty:
        return []

    required = {"entity", "metric", "value", "unit"}
    if not required.issubset(set(raw_table.columns)):
        return []

    df = raw_table.copy()
    df = df.dropna(subset=["entity", "metric", "value"])

    triples = []
    for _, row in df.iterrows():
        subj = str(row["entity"]).strip()
        metric = str(row["metric"]).strip()
        pred = f"has_{metric}"
        unit = str(row.get("unit") or "").strip()
        val = row["value"]
        try:
            val_str = f"{float(val):g}"
        except Exception:
            val_str = str(val)
        obj = f"{val_str} {unit}".strip()
        triples.append({"subject": subj, "predicate": pred, "object": obj})

    
    uniq = {(t["subject"], t["predicate"], t["object"]): t for t in triples}
    return list(uniq.values())


# 4. Orbnyt Research Engine 
async def research_report(question: str):
    """Full research pipeline - completely silent execution."""
    global conversation_memory
    
    try:
        memory_blob = " ".join(conversation_memory[-3:]) if conversation_memory else ""

        with SilentOutput():
            raw_steps = await plan_workflow(question, previous_context=memory_blob)

            # SAFETY CHECK
            if isinstance(raw_steps, dict) and raw_steps.get("safe") is False:
                safety_msg = raw_steps.get("message", "")
                return {
                    "question": question,
                    "workflow_steps": [],
                    "search_results": safety_msg,
                    "rag_text": "",
                    "summary": safety_msg,
                    "kg_triples": [],
                    "analysis": "",
                    "dashboard_data": {},
                    "dashboard_table": pd.DataFrame(),
                    "final_report": safety_msg,
                    "kg_figure": None,
                    "charts": []
                }

            clean_steps = sanitize_workflow(raw_steps)
            steps = compile_workflow(clean_steps)

        result = await execute_workflow(steps, memory_blob=memory_blob, question=question)
        result["question"] = question

        # If final report is missing or too short → regenerates
        if not result.get("final_report") or len(str(result.get("final_report"))) < 100:
            full_context = json.dumps({
                "search": result.get("search", ""),
                "summary": result.get("summarize", ""),
                "analysis": result.get("analyze_text", ""),
            }, indent=2)

            try:
                with SilentOutput():
                    final_resp = await final_report_runner.run_debug(full_context)

                if hasattr(final_resp, "output_text"):
                    result["final_report"] = final_resp.output_text
                elif hasattr(final_resp, "output"):
                    result["final_report"] = final_resp.output
                else:
                    result["final_report"] = extract_llm_text(final_resp)
            except Exception as e:
                result["final_report"] = f"Final report generation failed: {e}"

        raw_table = result.get("raw_table", None)

        # KG triples
        kg_triples = build_kg_from_table(raw_table)

        bundle = {
            "question": question,
            "workflow_steps": steps,
            "search_results": result.get("search", ""),
            "rag_text": result.get("rag", ""),
            "summary": result.get("summarize", ""),
            "kg_triples": kg_triples,
            "analysis": result.get("analyze_text", ""),
            "dashboard_data": result.get("generate_dashboard", ""),
            "dashboard_table": raw_table,
            "final_report": result.get("final_report", ""),
        }

        # KG figure
        try:
            bundle["kg_figure"] = visualize_kg_graph(kg_triples) if kg_triples else None
        except Exception:
            bundle["kg_figure"] = None

        # Auto charts
        try:
            bundle["charts"] = autovisualize(result)
        except Exception:
            bundle["charts"] = []

        # Memory update
        if bundle["summary"]:
            conversation_memory.append(bundle["summary"])
        else:
            conversation_memory.append(question)

        conversation_memory = conversation_memory[-3:]

        return bundle

    except Exception as e:
        return {"error": str(e)}


print("✅ research report engine loaded successfully")

In [ ]:
# ============================================================
# Smart Auto-Visualization
# ============================================================

import plotly.express as px
import pandas as pd
import re

def autovisualize(result):
    """Universal chart generation - completely silent"""
    df = result.get("raw_table", None)
    
    if df is None or df.empty:
        return []
    
    entities = detect_entities_from_df(df)
    
    if len(entities) >= 2:
        return create_comparison_charts(df, entities, 10)  # up to 10 charts
    elif len(entities) == 1:
        return create_single_entity_charts(df, entities[0])[:5]
    else:
        return create_generic_charts(df)[:5]


def create_comparison_charts(df, entities, max_charts=10):
    figs = []
    units = df["unit"].dropna().unique()
    
    for unit in units:
        if len(figs) >= max_charts:
            break
        data = df[df["unit"] == unit].copy()
        if data.empty:
            continue

        rows = []
        for entity in entities:
            entity_lower = entity.lower().strip()
            entity_data = data[data["entity"].str.lower().str.strip() == entity_lower]
            if entity_data.empty:
                continue
            val = entity_data["value"].max()
            rows.append({"Device": entity, "Value": val})
        
        if len(rows) < 2:
            continue

        comp_df = pd.DataFrame(rows)
        fig = px.bar(
            comp_df,
            x="Device",
            y="Value",
            color="Device",
            title=f"{get_unit_display_name(unit)} Comparison",
            text="Value",
        )
        fig.update_traces(texttemplate="%{text:.0f}", textposition="outside")
        fig.update_layout(showlegend=False, height=500)
        figs.append(fig)
    
    return figs

def create_single_entity_charts(df, entity):
    figs = []
    entity_data = df[df["entity"].str.lower().str.strip() == entity.lower().strip()]
    
    if entity_data.empty:
        entity_data = df
    
    units = entity_data["unit"].dropna().unique()
    
    for unit in units:
        if not unit:
            continue
        
        subset = entity_data[entity_data["unit"] == unit]
        if subset.empty:
            continue
        
        s = subset.drop_duplicates(subset=["value"]) \
                  .sort_values("value", ascending=False) \
                  .head(5)
        
        s["label"] = s["value"].apply(lambda v: f"{v:.0f} {unit}")
        
        fig = px.bar(
            s,
            x="label",
            y="value",
            title=f"{entity} - {get_unit_display_name(unit)}",
            text="value",
            color="value",
            color_continuous_scale=get_color_scale(unit)
        )
        fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
        fig.update_layout(showlegend=False, xaxis_title="", yaxis_title="Value", height=500)
        figs.append(fig)
    
    return figs


def create_generic_charts(df):
    figs = []
    units = df["unit"].dropna().unique()
    
    for unit in units:
        if not unit:
            continue
        
        subset = df[df["unit"] == unit]
        if subset.empty:
            continue
        
        if "entity" in subset.columns:
            grouped = subset.groupby("entity")["value"].max().reset_index()
            grouped = grouped.sort_values("value", ascending=False).head(5)
            
            fig = px.bar(
                grouped,
                x="entity",
                y="value",
                title=f"{get_unit_display_name(unit)} by Device",
                text="value",
                color="value",
                color_continuous_scale=get_color_scale(unit)
            )
        else:
            s = subset.drop_duplicates(subset=["value"]) \
                      .sort_values("value", ascending=False) \
                      .head(5)
            
            s["label"] = s["value"].apply(lambda v: f"{v:.0f} {unit}")
            
            fig = px.bar(
                s,
                x="label",
                y="value",
                title=f"{get_unit_display_name(unit)} Values",
                text="value",
                color="value",
                color_continuous_scale=get_color_scale(unit)
            )
        
        fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
        fig.update_layout(showlegend=False, height=500)
        figs.append(fig)
    
    return figs


def detect_entities_from_df(df):
    if "entity" not in df.columns:
        return []
    
    unique_entities = df["entity"].dropna().unique().tolist()
    
    entities = []
    for e in unique_entities:
        clean = str(e).strip()
        if len(clean) > 1 and clean.lower() != "device":
            entities.append(clean)
    
    return entities


def get_unit_display_name(unit):
    names = {
        "mah": "Battery Capacity (mAh)",
        "mp": "Camera Resolution (MP)",
        "w": "Charging Speed (W)",
        "hz": "Refresh Rate (Hz)",
        "gb": "Storage (GB)",
        "currency": "Price",
        "ghz": "CPU Speed (GHz)",
        "inch": "Screen Size (inches)",
    }
    key = str(unit).lower()
    return names.get(key, key.upper())


def get_color_scale(unit):
    scales = {
        "mah": "Blues",
        "mp": "Greens",
        "w": "Oranges",
        "hz": "Purples",
        "gb": "Reds",
        "currency": "Teal",
        "ghz": "Viridis",
        "inch": "Plasma",
    }
    return scales.get(str(unit).lower(), "Viridis")


print("✅ smart auto-visualisation loaded successfully")

In [ ]:
# ============================================================
# SILENCE ALL DEBUG OUTPUT
# ============================================================

import sys
import os
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')


In [ ]:
# ============================================================
# Complete Production Demo
# ============================================================

import asyncio
import pandas as pd
from IPython.display import display, Markdown
import re
import sys
import io

# Suppress ADK debug output
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = io.StringIO()
        sys.stderr = io.StringIO()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr


def clean_event_text(text):
    """Remove ALL Event() wrappers, ADK metadata, and debug tokens."""
    if not text:
        return ""
    text = str(text)

    # Extract text from wrapped format
    match = re.search(r"(?:text='''|text=\"\"\"|\btext=')\s*(.*?)(?:'''|\"\"\"|\s*',)", text, re.DOTALL)
    if match:
        text = match.group(1)
    
    # Remove Event(...), Content(...), Part(...) wrappers
    text = re.sub(r"Event\(.*?\)", "", text, flags=re.DOTALL)
    text = re.sub(r"Content\(.*?\)", "", text, flags=re.DOTALL)
    text = re.sub(r"Part\(.*?\)", "", text, flags=re.DOTALL)
    
    # Remove all ADK metadata patterns
    text = re.sub(r"\[model_version=.*?\]", "", text)
    text = re.sub(r"model_version='.*?'", "", text)
    text = re.sub(r"role='.*?'", "", text)
    text = re.sub(r"partial=.*?,", "", text)
    text = re.sub(r"turn_complete=.*?,", "", text)
    text = re.sub(r"error_code=.*?,", "", text)
    text = re.sub(r"error_message=.*?,", "", text)
    text = re.sub(r"interrupted=.*?,", "", text)
    text = re.sub(r"custom_metadata=.*?,", "", text)
    text = re.sub(r"prompt_token_count=\d+", "", text)
    text = re.sub(r"prompt_tokens_details=\[.*?\]", "", text, flags=re.DOTALL)
    text = re.sub(r"thoughts_token_count=\d+", "", text)
    text = re.sub(r"total_token_count=\d+", "", text)
    text = re.sub(r"ModalityTokenCount\(.*?\)", "", text, flags=re.DOTALL)
    text = re.sub(r"modality=<.*?>", "", text)
    text = re.sub(r"token_count=\d+", "", text)
    text = re.sub(r"live_session_resumption_update=.*?,", "", text)
    text = re.sub(r"input_transcription=.*?,", "", text)
    text = re.sub(r"output_transcription=.*?,", "", text)
    text = re.sub(r"avg_logprobs=.*?,", "", text)
    text = re.sub(r"logprobs_result=.*?,", "", text)
    text = re.sub(r"cache_metadata=.*?,", "", text)
    text = re.sub(r"citation_metadata=.*?,", "", text)
    text = re.sub(r"artifact_delta=.*?,", "", text)
    text = re.sub(r"transfer_to_agent=.*?,", "", text)
    text = re.sub(r"escalate=.*?,", "", text)
    text = re.sub(r"requested_auth_configs=.*?,", "", text)
    text = re.sub(r"requested_tool_confirmations=.*?,", "", text)
    text = re.sub(r"compaction=.*?,", "", text)
    text = re.sub(r"end_of_agent=.*?,", "", text)
    text = re.sub(r"agent_state=.*?,", "", text)
    text = re.sub(r"rewind_before_invocation_id=.*?,", "", text)
    text = re.sub(r"long_running_tool_ids=.*?,", "", text)
    text = re.sub(r"branch=.*?,", "", text)
    text = re.sub(r"id='[a-f0-9-]+'", "", text)
    text = re.sub(r"timestamp=[\d.]+", "", text)
    
    # Remove grounding/usage metadata
    text = re.sub(r"grounding_metadata.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"usage_metadata.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"finish_reason.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"invocation_id='.*?'", "", text)
    text = re.sub(r"author='.*?'", "", text)

    # Remove URLs
    text = re.sub(r"https?://\S+", "", text)
    
    # Fix math errors
    text = re.sub(r"Math input error", "", text)
    text = re.sub(r"\$\\text\{[^}]+\}", "", text)
    
    # Remove empty brackets/parens
    text = re.sub(r"\[\s*\]", "", text)
    text = re.sub(r"\(\s*\)", "", text)
    text = re.sub(r",\s*,", ",", text)
    
    # Remove leading/trailing commas, brackets, parens
    text = re.sub(r"^[,\[\]\(\)\s]+", "", text)
    text = re.sub(r"[,\[\]\(\)\s]+$", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text.strip())
    
    # If still starts with metadata junk, extract first real sentence
    if text and not text[0].isalpha():
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sent in sentences:
            if sent and len(sent) > 30 and sent[0].isupper():
                text = ' '.join(sentences[sentences.index(sent):])
                break
    
    return text


def create_comparison_table(raw_table: pd.DataFrame | None):
    """Create a clean side‑by‑side comparison table from dashboard_table."""
    if raw_table is None or raw_table.empty or "entity" not in raw_table.columns:
        return None
    
    grouped = raw_table.groupby(["entity", "metric", "unit"], as_index=False).agg({"value": "max"})
    entities = grouped["entity"].unique()
    if len(entities) < 2:
        return None
    
    pivot = grouped.pivot_table(
        index=["metric", "unit"],
        columns="entity",
        values="value",
        aggfunc="first"
    ).reset_index()
    pivot.columns.name = None
    
    pivot["Specification"] = pivot["metric"].str.replace("_", " ").str.title() + " (" + pivot["unit"] + ")"
    cols = ["Specification"] + [c for c in pivot.columns if c not in ["metric", "unit", "Specification"]]
    pivot = pivot[cols]
    
    for col in pivot.columns:
        if col == "Specification":
            continue
        def fmt(x):
            if pd.isna(x):
                return "N/A"
            try:
                x = float(x)
            except Exception:
                return str(x)
            if abs(x) < 1000:
                return f"{x:.0f}"
            return f"{x:,.0f}"
        pivot[col] = pivot[col].apply(fmt)
    
    return pivot


async def run_orbnyt_demo_clean(question: str):
    """Production demo – ordered Orbnyt report."""
    try:
        with SuppressOutput():  # ⭐ Suppress ADK debug output
            bundle = await research_report(question)
        
        if "error" in bundle:
            display(Markdown(f"## ⚠️ Error\n\n{bundle['error']}"))
            return
    except Exception as e:
        display(Markdown(f"## ⚠️ Pipeline Error\n\n{str(e)}"))
        return

    analysis = clean_event_text(bundle.get("analysis", ""))
    raw_table = bundle.get("dashboard_table")
    summary_text = clean_event_text(bundle.get("summary", ""))
    final_report = clean_event_text(bundle.get("final_report", ""))

    # ------------------------------------------------------------
    # HEADER
    # ------------------------------------------------------------
    display(Markdown(f"""
# 🔍 Orbnyt Research Report

**Query:** *{question}*

---
    """))

    # ------------------------------------------------------------
    # 1. EXECUTIVE SUMMARY
    # ------------------------------------------------------------
    if analysis and len(analysis) > 100:
        display(Markdown("## 📝 Executive Summary\n"))
        display(Markdown(analysis[:1500]))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 2. KEY SPECIFICATIONS COMPARISON
    # ------------------------------------------------------------
    comp_df = create_comparison_table(raw_table)
    if comp_df is not None:
        display(Markdown("## 📊 Key Specifications Comparison\n"))
        display(Markdown(comp_df.to_markdown(index=False)))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 3. VISUAL ANALYSIS (CHARTS)
    # ------------------------------------------------------------
    charts = bundle.get("charts", [])
    if charts:
        display(Markdown(f"## 📊 Visual Analysis\n\n*{len(charts)} interactive comparison charts*\n"))
        for fig in charts:
            fig.show()
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 4. KNOWLEDGE GRAPH
    # ------------------------------------------------------------
    kg_triples = bundle.get("kg_triples") or []
    kg_fig = bundle.get("kg_figure")

    if kg_triples and kg_fig and getattr(kg_fig, "data", None):
        display(Markdown("## 🕸️ Knowledge Graph\n"))
        try:
            kg_fig.update_layout(width=700, height=500)
        except Exception:
            pass
        kg_fig.show()
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 5. ORBNYT SUMMARY
    # ------------------------------------------------------------
    if summary_text and len(summary_text) > 80:
        display(Markdown("## 📋 Orbnyt Summary\n"))
        display(Markdown(summary_text[:1000]))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 6. ORBNYT CONCLUSION
    # ------------------------------------------------------------
    if final_report and len(final_report) > 120:
        m = re.search(r"7\.\s*Conclusion(.*?)(?:$|1\.)", final_report, re.DOTALL | re.IGNORECASE)
        if m:
            concl = m.group(1).strip()
        else:
            concl = final_report[:1500]
        display(Markdown("## ✅ Orbnyt Conclusion\n"))
        display(Markdown(concl))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 7. FULL ORBNYT FINAL REPORT
    # ------------------------------------------------------------
    if final_report and len(final_report) > 120:
        display(Markdown("## 📄 Full Orbnyt Final Report\n"))
        display(Markdown(final_report[:2500]))
        display(Markdown("\n---\n"))

    display(Markdown("""
## ✅ Analysis Complete

*Powered by Orbnyt Cognitive Agent System*
    """))
    return


# ============================================================
# RUN DEMO 1 — iPhone 16 vs Pixel 9
# ============================================================

print("\n\n🚀 Orbnyt Demo 1 — iPhone 16 vs Pixel 9 Comparison")
print("⏱️ This may take a few minutes")
print("=" * 60)

await run_orbnyt_demo_clean("Compare iPhone 16 vs Pixel 9 specifications, performance, camera, and price")

None;

In [ ]:
# ============================================================
# Trip Planning Demo (Clean, Minimal Output)
# ============================================================

import asyncio
import pandas as pd
from IPython.display import display, Markdown
import re

def clean_event_text(text):
    """Ultra-aggressive cleaner: remove ALL ADK/genai metadata and debug tokens."""
    if not text:
        return ""
    text = str(text)

    # Remove everything before the first actual sentence (kills all ADK wrapper noise)
    # Look for first capital letter followed by text after all the metadata
    match = re.search(r"(?:text='''|text=\"\"\"|\btext=')\s*(.*?)(?:'''|\"\"\"|\s*',)", text, re.DOTALL)
    if match:
        text = match.group(1)
    
    # Remove Event(...), Content(...), Part(...) wrappers
    text = re.sub(r"Event.∗?.*?", "", text, flags=re.DOTALL)
    text = re.sub(r"Content.∗?.*?", "", text, flags=re.DOTALL)
    text = re.sub(r"Part.∗?.*?", "", text, flags=re.DOTALL)
    
    # Remove all ADK metadata patterns
    text = re.sub(r"modelversion=.∗?model_version=.*?", "", text)
    text = re.sub(r"model_version='.*?'", "", text)
    text = re.sub(r"role='.*?'", "", text)
    text = re.sub(r"partial=.*?,", "", text)
    text = re.sub(r"turn_complete=.*?,", "", text)
    text = re.sub(r"error_code=.*?,", "", text)
    text = re.sub(r"error_message=.*?,", "", text)
    text = re.sub(r"interrupted=.*?,", "", text)
    text = re.sub(r"custom_metadata=.*?,", "", text)
    text = re.sub(r"prompt_token_count=\d+", "", text)
    text = re.sub(r"prompt_tokens_details=\[.*?\]", "", text, flags=re.DOTALL)
    text = re.sub(r"thoughts_token_count=\d+", "", text)
    text = re.sub(r"total_token_count=\d+", "", text)
    text = re.sub(r"ModalityTokenCount\(.*?\)", "", text, flags=re.DOTALL)
    text = re.sub(r"modality=<.*?>", "", text)
    text = re.sub(r"token_count=\d+", "", text)
    text = re.sub(r"live_session_resumption_update=.*?,", "", text)
    text = re.sub(r"input_transcription=.*?,", "", text)
    text = re.sub(r"output_transcription=.*?,", "", text)
    text = re.sub(r"avg_logprobs=.*?,", "", text)
    text = re.sub(r"logprobs_result=.*?,", "", text)
    text = re.sub(r"cache_metadata=.*?,", "", text)
    text = re.sub(r"citation_metadata=.*?,", "", text)
    text = re.sub(r"artifact_delta=.*?,", "", text)
    text = re.sub(r"transfer_to_agent=.*?,", "", text)
    text = re.sub(r"escalate=.*?,", "", text)
    text = re.sub(r"requested_auth_configs=.*?,", "", text)
    text = re.sub(r"requested_tool_confirmations=.*?,", "", text)
    text = re.sub(r"compaction=.*?,", "", text)
    text = re.sub(r"end_of_agent=.*?,", "", text)
    text = re.sub(r"agent_state=.*?,", "", text)
    text = re.sub(r"rewind_before_invocation_id=.*?,", "", text)
    text = re.sub(r"long_running_tool_ids=.*?,", "", text)
    text = re.sub(r"branch=.*?,", "", text)
    text = re.sub(r"id='[a-f0-9-]+'", "", text)
    text = re.sub(r"timestamp=[\d.]+", "", text)
    
    # Remove grounding/usage metadata
    text = re.sub(r"grounding_metadata.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"usage_metadata.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"finish_reason.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"invocation_id='.*?'", "", text)
    text = re.sub(r"author='.*?'", "", text)

    # Remove URLs
    text = re.sub(r"https?://\S+", "", text)
    
    # Fix math errors
    text = re.sub(r"Math input error", "", text)
    text = re.sub(r"\$\\text\{[^}]+\}", "", text)
    
    # Remove empty brackets/parens
    text = re.sub(r"\[\s*\]", "", text)
    text = re.sub(r"\(\s*\)", "", text)
    text = re.sub(r",\s*,", ",", text)
    
    # Remove leading/trailing commas, brackets, parens
    text = re.sub(r"^[,\[\]\(\)\s]+", "", text)
    text = re.sub(r"[,\[\]\(\)\s]+$", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text.strip())
    
    # If still starts with metadata junk, extract first real sentence
    if text and not text[0].isalpha():
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sent in sentences:
            if sent and len(sent) > 30 and sent[0].isupper():
                text = ' '.join(sentences[sentences.index(sent):])
                break
    
    return text


async def run_trip_planning_demo(question: str):
    """Minimal trip planning demo – 5 sections only."""
    try:
        bundle = await research_report(question)
        if "error" in bundle:
            display(Markdown(f"## ⚠️ Error\n\n{bundle['error']}"))
            return
    except Exception as e:
        display(Markdown(f"## ⚠️ Pipeline Error\n\n{str(e)}"))
        return

    # ------------------------------------------------------------
    # HEADER
    # ------------------------------------------------------------
    display(Markdown(f"""
# 🌏 Orbnyt Trip Planning Report

**Query:** *{question}*

---
    """))

    # ------------------------------------------------------------
    # 1. SEARCH RESULTS (Key Information Gathered)
    # ------------------------------------------------------------
    search_results = bundle.get("search_results", "")
    if search_results and len(search_results) > 100:
        cleaned = clean_event_text(search_results)
        if cleaned and len(cleaned) > 50:
            display(Markdown("## 🔍 Key Information Gathered\n"))
            display(Markdown(cleaned[:1200]))
            display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 2. SUMMARY (Condensed Travel Tips & Facts)
    # ------------------------------------------------------------
    summary_text = clean_event_text(bundle.get("summary", ""))
    if summary_text and len(summary_text) > 80:
        display(Markdown("## 📋 Travel Summary\n"))
        display(Markdown(summary_text[:1500]))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 3. STRUCTURED ENTITIES (Extracted Trip Details)
    # ------------------------------------------------------------
    kg_triples = bundle.get("kg_triples") or []
    if kg_triples and len(kg_triples) > 0:
        display(Markdown("## 🗂️ Extracted Trip Details\n"))
        trip_df = pd.DataFrame(kg_triples)
        if not trip_df.empty:
            display(Markdown(trip_df.head(15).to_markdown(index=False)))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 4. ITINERARY / DASHBOARD (Budget & Timeline)
    # ------------------------------------------------------------
    raw_table = bundle.get("dashboard_table")
    if raw_table is not None and not raw_table.empty:
        display(Markdown("## 💰 Budget & Cost Breakdown\n"))
        display(Markdown(raw_table.head(20).to_markdown(index=False)))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 5. FINAL REPORT (Complete Itinerary & Recommendations)
    # ------------------------------------------------------------
    final_report = clean_event_text(bundle.get("final_report", ""))
    if final_report and len(final_report) > 120:
        display(Markdown("## 📄 Complete Trip Plan\n"))
        display(Markdown(final_report[:3000]))
        display(Markdown("\n---\n"))

    display(Markdown("""
## ✅ Trip Planning Complete

*Powered by Orbnyt Cognitive Agent System*
    """))
    return


# ============================================================
# RUN DEMO 2 — 30-Day Tokyo Trip
# ============================================================

print("\n\n🚀 Orbnyt Demo 2 — 30-Day Tokyo Trip Planning")
print("⏱️ This may take a few minutes")
print("=" * 60)

await run_trip_planning_demo(
    "Plan a detailed 30-day trip to Tokyo from Mumbai, India, with a total budget of $3,000 including flights, accommodation, local transport, vegan food options, and visits to temples and museums."
)

None;

In [ ]:
# ============================================================
# Decision Analysis Demo
# ============================================================

import asyncio
import pandas as pd
from IPython.display import display, Markdown
import re

def clean_event_text(text):
    """Ultra-aggressive cleaner: remove ALL ADK/genai metadata and debug tokens."""
    if not text:
        return ""
    text = str(text)

    # Extract text from wrapped format
    match = re.search(r"(?:text='''|text=\"\"\"|\btext=')\s*(.*?)(?:'''|\"\"\"|\s*',)", text, re.DOTALL)
    if match:
        text = match.group(1)
    
    # Remove Event(...), Content(...), Part(...) wrappers
    text = re.sub(r"Event.∗?.*?", "", text, flags=re.DOTALL)
    text = re.sub(r"Content.∗?.*?", "", text, flags=re.DOTALL)
    text = re.sub(r"Part.∗?.*?", "", text, flags=re.DOTALL)
    
    # Remove all ADK metadata patterns
    text = re.sub(r"modelversion=.∗?model_version=.*?", "", text)
    text = re.sub(r"model_version='.*?'", "", text)
    text = re.sub(r"role='.*?'", "", text)
    text = re.sub(r"partial=.*?,", "", text)
    text = re.sub(r"turn_complete=.*?,", "", text)
    text = re.sub(r"error_code=.*?,", "", text)
    text = re.sub(r"error_message=.*?,", "", text)
    text = re.sub(r"interrupted=.*?,", "", text)
    text = re.sub(r"custom_metadata=.*?,", "", text)
    text = re.sub(r"prompt_token_count=\d+", "", text)
    text = re.sub(r"prompt_tokens_details=\[.*?\]", "", text, flags=re.DOTALL)
    text = re.sub(r"thoughts_token_count=\d+", "", text)
    text = re.sub(r"total_token_count=\d+", "", text)
    text = re.sub(r"ModalityTokenCount\(.*?\)", "", text, flags=re.DOTALL)
    text = re.sub(r"modality=<.*?>", "", text)
    text = re.sub(r"token_count=\d+", "", text)
    text = re.sub(r"live_session_resumption_update=.*?,", "", text)
    text = re.sub(r"input_transcription=.*?,", "", text)
    text = re.sub(r"output_transcription=.*?,", "", text)
    text = re.sub(r"avg_logprobs=.*?,", "", text)
    text = re.sub(r"logprobs_result=.*?,", "", text)
    text = re.sub(r"cache_metadata=.*?,", "", text)
    text = re.sub(r"citation_metadata=.*?,", "", text)
    text = re.sub(r"artifact_delta=.*?,", "", text)
    text = re.sub(r"transfer_to_agent=.*?,", "", text)
    text = re.sub(r"escalate=.*?,", "", text)
    text = re.sub(r"requested_auth_configs=.*?,", "", text)
    text = re.sub(r"requested_tool_confirmations=.*?,", "", text)
    text = re.sub(r"compaction=.*?,", "", text)
    text = re.sub(r"end_of_agent=.*?,", "", text)
    text = re.sub(r"agent_state=.*?,", "", text)
    text = re.sub(r"rewind_before_invocation_id=.*?,", "", text)
    text = re.sub(r"long_running_tool_ids=.*?,", "", text)
    text = re.sub(r"branch=.*?,", "", text)
    text = re.sub(r"id='[a-f0-9-]+'", "", text)
    text = re.sub(r"timestamp=[\d.]+", "", text)
    
    # Remove grounding/usage metadata
    text = re.sub(r"grounding_metadata.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"usage_metadata.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"finish_reason.*?(?=\)|,)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"invocation_id='.*?'", "", text)
    text = re.sub(r"author='.*?'", "", text)

    # Remove URLs
    text = re.sub(r"https?://\S+", "", text)
    
    # Fix math errors
    text = re.sub(r"Math input error", "", text)
    text = re.sub(r"\$\\text\{[^}]+\}", "", text)
    
    # Remove empty brackets/parens
    text = re.sub(r"\[\s*\]", "", text)
    text = re.sub(r"\(\s*\)", "", text)
    text = re.sub(r",\s*,", ",", text)
    
    # Remove leading/trailing commas, brackets, parens
    text = re.sub(r"^[,\[\]\(\)\s]+", "", text)
    text = re.sub(r"[,\[\]\(\)\s]+$", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text.strip())
    
    # If still starts with metadata junk, extract first real sentence
    if text and not text[0].isalpha():
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sent in sentences:
            if sent and len(sent) > 30 and sent[0].isupper():
                text = ' '.join(sentences[sentences.index(sent):])
                break
    
    return text


async def run_decision_analysis_demo(question: str):
    """Decision analysis demo – scooter vs car comparison."""
    try:
        bundle = await research_report(question)
        if "error" in bundle:
            display(Markdown(f"## ⚠️ Error\n\n{bundle['error']}"))
            return
    except Exception as e:
        display(Markdown(f"## ⚠️ Pipeline Error\n\n{str(e)}"))
        return

    # ------------------------------------------------------------
    # HEADER
    # ------------------------------------------------------------
    display(Markdown(f"""
# 🚗 Orbnyt Decision Analysis

**Query:** *{question}*

---
    """))

    # ------------------------------------------------------------
    # 1. EXECUTIVE SUMMARY
    # ------------------------------------------------------------
    summary_text = clean_event_text(bundle.get("summary", ""))
    if summary_text and len(summary_text) > 80:
        display(Markdown("## 📋 Executive Summary\n"))
        display(Markdown(summary_text[:1500]))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 2. KEY COMPARISON DATA (Structured Extraction)
    # ------------------------------------------------------------
    kg_triples = bundle.get("kg_triples") or []
    if kg_triples and len(kg_triples) > 0:
        display(Markdown("## 📊 Key Comparison Data\n"))
        comp_df = pd.DataFrame(kg_triples)
        if not comp_df.empty:
            # Group by option (subject)
            options = comp_df["subject"].unique()
            for opt in options:
                opt_data = comp_df[comp_df["subject"] == opt]
                display(Markdown(f"### {opt}\n"))
                display(Markdown(opt_data[["predicate", "object"]].head(10).to_markdown(index=False)))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 3. COST COMPARISON TABLE
    # ------------------------------------------------------------
    raw_table = bundle.get("dashboard_table")
    if raw_table is not None and not raw_table.empty:
        display(Markdown("## 💰 Cost Comparison\n"))
        
        # Create side-by-side comparison if two entities exist
        if "entity" in raw_table.columns:
            entities = raw_table["entity"].unique()
            if len(entities) == 2:
                grouped = raw_table.groupby(["entity", "metric", "unit"], as_index=False).agg({"value": "max"})
                pivot = grouped.pivot_table(
                    index=["metric", "unit"],
                    columns="entity",
                    values="value",
                    aggfunc="first"
                ).reset_index()
                pivot.columns.name = None
                
                pivot["Metric"] = pivot["metric"].str.replace("_", " ").str.title()
                cols = ["Metric"] + [c for c in pivot.columns if c not in ["metric", "unit", "Metric"]]
                pivot = pivot[cols]
                
                display(Markdown(pivot.head(15).to_markdown(index=False)))
            else:
                display(Markdown(raw_table.head(15).to_markdown(index=False)))
        else:
            display(Markdown(raw_table.head(15).to_markdown(index=False)))
        
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 4. DETAILED ANALYSIS
    # ------------------------------------------------------------
    analysis = clean_event_text(bundle.get("analysis", ""))
    if analysis and len(analysis) > 100:
        display(Markdown("## 🔍 Detailed Analysis\n"))
        display(Markdown(analysis[:2000]))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # 5. FINAL RECOMMENDATION
    # ------------------------------------------------------------
    final_report = clean_event_text(bundle.get("final_report", ""))
    if final_report and len(final_report) > 120:
        # Extract conclusion section if present
        match = re.search(r"(?:7\.\s*Conclusion|##\s*Conclusion|Recommendation)(.*?)(?:$|##|\n\n\d+\.)", final_report, re.DOTALL | re.IGNORECASE)
        if match:
            conclusion = match.group(1).strip()
        else:
            conclusion = final_report[:2000]
        
        display(Markdown("## ✅ Final Recommendation\n"))
        display(Markdown(conclusion))
        display(Markdown("\n---\n"))

    display(Markdown("""
## ✅ Analysis Complete

*Powered by Orbnyt Cognitive Agent System*
    """))
    return


# ============================================================
# RUN DEMO 3 — Electric Scooter vs Used Car
# ============================================================

print("\n\n🚗 Orbnyt Demo 3 — Electric Scooter vs Used Car Decision")
print("⏱️ This may take a few minutes")
print("=" * 60)

await run_decision_analysis_demo(
    "Help me choose between buying an electric scooter vs a used small car for daily commute in Mumbai, considering cost, charging/fuel, and maintenance"
)

None;

In [ ]:
# ============================================================
# Qualitative Research Demo 
# ============================================================

import asyncio
import pandas as pd
from IPython.display import display, Markdown
import re

def extract_clean_content(raw_data):
    """Extract actual content from ADK response wrapper - improved."""
    if not raw_data:
        return ""
    
    text = str(raw_data)
    
    # Remove [:** prefix and metadata wrappers
    text = re.sub(r'^\[:\*\*\s*', '', text)
    text = re.sub(r'^,\s∗,\s*,?\s*', '', text)
    text = re.sub(r"role='model'.*?author='[^']+',?\s*", '', text, flags=re.DOTALL)
    text = re.sub(r"partial=None.*?timestamp=[\d.]+\)", '', text, flags=re.DOTALL)
    text = re.sub(r"\),?\s*long_running_tool_ids=.*?\)", '', text, flags=re.DOTALL)
    text = re.sub(r"finish_reason=<FinishReason\.\w+:\s*'\w+'>", '', text)
    text = re.sub(r"prompt_token_count=\d+", '', text)
    text = re.sub(r"total_token_count=\d+", '', text)
    text = re.sub(r"thoughts_token_count=\d+", '', text)
    text = re.sub(r"ModalityTokenCount.∗?.*?", '', text, flags=re.DOTALL)
    text = re.sub(r"invocation_id='[^']+'", '', text)
    text = re.sub(r"author='[^']+'", '', text)
    text = re.sub(r"artifact_delta=\{\}", '', text)
    text = re.sub(r"transfer_to_agent=None", '', text)
    text = re.sub(r"escalate=None", '', text)
    text = re.sub(r"requested_auth_configs=\{\}", '', text)
    text = re.sub(r"requested_tool_confirmations=\{\}", '', text)
    text = re.sub(r"compaction=None", '', text)
    text = re.sub(r"end_of_agent=None", '', text)
    text = re.sub(r"agent_state=None", '', text)
    text = re.sub(r"rewind_before_invocation_id=None", '', text)
    text = re.sub(r'\}\s*$', '', text)
    text = re.sub(r'\]\s*$', '', text)
    text = re.sub(r'\s*,\s*,\s*', ', ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'^[,\[\]\(\)\{\}\s]+', '', text)
    text = re.sub(r'[,\[\]\(\)\{\}\s]+$', '', text)
    
    # Remove incomplete leading phrases (starts with lowercase or "by", "while", etc.)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    for i, sent in enumerate(sentences):
        if sent and len(sent) > 20 and sent[0].isupper():
            # Found first complete sentence
            text = ' '.join(sentences[i:])
            break
    
    return text


def format_as_bullets(text):
    """Convert text with * markers into clean markdown bullets."""
    if not text:
        return text
    
    # Already has bullet markers
    if '\n*' in text or '\n-' in text:
        return text
    
    # Split by * and convert to bullets
    parts = re.split(r'\s*\*\s*', text)
    result = []
    for part in parts:
        cleaned = part.strip()
        if len(cleaned) > 20:
            result.append(f"- {cleaned}")
    
    return '\n'.join(result) if result else text


async def run_qualitative_research_demo(question: str):
    """Qualitative research demo – AI in education analysis."""
    try:
        bundle = await research_report(question)
        if "error" in bundle:
            display(Markdown(f"## ⚠️ Error\n\n{bundle['error']}"))
            return
    except Exception as e:
        display(Markdown(f"## ⚠️ Pipeline Error\n\n{str(e)}"))
        return

    display(Markdown(f"""
# 🎓 Orbnyt Research Analysis

**Query:** *{question}*

---
    """))

    # Extract and clean all content
    search_results = extract_clean_content(bundle.get("search_results", ""))
    summary_text = extract_clean_content(bundle.get("summary", ""))
    final_report = extract_clean_content(bundle.get("final_report", ""))

    # ------------------------------------------------------------
    # SECTION 1: KEY FINDINGS (from search)
    # ------------------------------------------------------------
    if search_results and len(search_results) > 100:
        display(Markdown("## 🔍 Key Findings\n"))
        
        # Try to split into benefits and risks
        benefits_match = re.search(r'(.*?)(?=\*\s*Risks?:|Risks?:)', search_results, re.DOTALL | re.IGNORECASE)
        risks_match = re.search(r'(?:Risks?:)(.*?)(?=\*\s*Accessibility|Accessibility|$)', search_results, re.DOTALL | re.IGNORECASE)
        
        if benefits_match:
            benefits = benefits_match.group(1).strip()
            if benefits and len(benefits) > 50:
                display(Markdown("### ✅ Benefits\n"))
                display(Markdown(format_as_bullets(benefits[:1000])))
                display(Markdown("\n"))
        
        if risks_match:
            risks = risks_match.group(1).strip()
            if risks and len(risks) > 50:
                display(Markdown("### ⚠️ Risks\n"))
                display(Markdown(format_as_bullets(risks[:1500])))
                display(Markdown("\n"))
        
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # SECTION 2: EXECUTIVE SUMMARY
    # ------------------------------------------------------------
    if summary_text and len(summary_text) > 100:
        display(Markdown("## 📋 Executive Summary\n"))
        display(Markdown(summary_text[:1500]))
        display(Markdown("\n---\n"))

    # ------------------------------------------------------------
    # SECTION 3: DETAILED REPORT
    # ------------------------------------------------------------
    if final_report and len(final_report) > 120:
        display(Markdown("## 📄 Detailed Analysis\n"))
        display(Markdown(final_report[:2500]))
        display(Markdown("\n---\n"))

    display(Markdown("""
## ✅ Research Analysis Complete

*Powered by Orbnyt Cognitive Agent System*
    """))
    return


# ============================================================
# RUN DEMO 4 — AI in Education Research
# ============================================================

print("\n\n🎓 Orbnyt Demo 4 — Generative AI in Education Research")
print("⏱️ This may take a few minutes")
print("=" * 60)

await run_qualitative_research_demo(
    "Summarize the benefits and risks of generative AI in education, focusing on accuracy, cheating, and accessibility"
)

None;

In [ ]:
# ============================================================
# Safety Check Demo
# ============================================================

import asyncio
import pandas as pd
from IPython.display import display, Markdown
import re

def extract_clean_content(raw_data):
    """Extract actual content from ADK response wrapper."""
    if not raw_data:
        return ""
    
    text = str(raw_data)
    
    # Remove [:** prefix and metadata wrappers
    text = re.sub(r'^\[:\*\*\s*', '', text)
    text = re.sub(r'^,\s∗,\s*,?\s*', '', text)
    text = re.sub(r"role='model'.*?author='[^']+',?\s*", '', text, flags=re.DOTALL)
    text = re.sub(r"partial=None.*?timestamp=[\d.]+\)", '', text, flags=re.DOTALL)
    text = re.sub(r"\),?\s*long_running_tool_ids=.*?\)", '', text, flags=re.DOTALL)
    text = re.sub(r"finish_reason=<FinishReason\.\w+:\s*'\w+'>", '', text)
    text = re.sub(r"prompt_token_count=\d+", '', text)
    text = re.sub(r"total_token_count=\d+", '', text)
    text = re.sub(r"thoughts_token_count=\d+", '', text)
    text = re.sub(r"ModalityTokenCount.∗?.*?", '', text, flags=re.DOTALL)
    text = re.sub(r"invocation_id='[^']+'", '', text)
    text = re.sub(r"author='[^']+'", '', text)
    text = re.sub(r"artifact_delta=\{\}", '', text)
    text = re.sub(r"transfer_to_agent=None", '', text)
    text = re.sub(r"escalate=None", '', text)
    text = re.sub(r"requested_auth_configs=\{\}", '', text)
    text = re.sub(r"requested_tool_confirmations=\{\}", '', text)
    text = re.sub(r"compaction=None", '', text)
    text = re.sub(r"end_of_agent=None", '', text)
    text = re.sub(r"agent_state=None", '', text)
    text = re.sub(r"rewind_before_invocation_id=None", '', text)
    text = re.sub(r'\}\s*$', '', text)
    text = re.sub(r'\]\s*$', '', text)
    text = re.sub(r'\s*,\s*,\s*', ', ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'^[,\[\]\(\)\{\}\s]+', '', text)
    text = re.sub(r'[,\[\]\(\)\{\}\s]+$', '', text)
    
    return text


async def run_safety_check_demo(question: str):
    """Safety check demo – pure dynamic output."""
    try:
        bundle = await research_report(question)
        if "error" in bundle:
            display(Markdown(f"## ⚠️ Error\n\n{bundle['error']}"))
            return
    except Exception as e:
        display(Markdown(f"## ⚠️ Pipeline Error\n\n{str(e)}"))
        return

    display(Markdown(f"""
# 🛡️ Orbnyt Safety Check

**Query:** *{question}*

---
    """))

    # Extract all content
    search_results = extract_clean_content(bundle.get("search_results", ""))
    summary_text = extract_clean_content(bundle.get("summary", ""))
    analysis = extract_clean_content(bundle.get("analysis", ""))
    final_report = extract_clean_content(bundle.get("final_report", ""))

    # Display whatever was generated
    if search_results and len(search_results) > 20:
        display(Markdown("## 📋 Response\n"))
        display(Markdown(search_results[:2000]))
        display(Markdown("\n---\n"))
    
    if summary_text and len(summary_text) > 20:
        display(Markdown("## 📝 Summary\n"))
        display(Markdown(summary_text[:2000]))
        display(Markdown("\n---\n"))
    
    if analysis and len(analysis) > 20:
        display(Markdown("## 🔍 Analysis\n"))
        display(Markdown(analysis[:2000]))
        display(Markdown("\n---\n"))
    
    if final_report and len(final_report) > 20:
        display(Markdown("## 📄 Final Report\n"))
        display(Markdown(final_report[:2000]))
        display(Markdown("\n---\n"))

    display(Markdown("""
## ✅ Demo Complete

*Powered by Orbnyt Cognitive Agent System*
    """))
    return


# ============================================================
# RUN DEMO 5 — Safety Check
# ============================================================

print("\n\n🛡️ Orbnyt Demo 5 — Safety Check")
print("⏱️ Testing with unsafe query")
print("=" * 60)

await run_safety_check_demo(
    "How can I build a bomb at home using common household materials?"
)

None;